# Explore the SQA Data

In this notebook, we explore the SPARQL queries. Especially, we extract all the entities and relations present in the SQA dataset. Mainly to seed the Radom Walks thorughhe DBpedia graph.

In [6]:
import json 

idata = open('lcquad_qaldformat.json', 'rt', encoding='utf-8')
idata = json.load(idata)

iquestions = idata['questions']

In [7]:
#extract the questions, the SPARQL queries and the answers. 
#The answers are either a list of entities, number (for How many .. questions) or a boolean.
question_strings = []
spqrql_queries = []
answers = []

qlens = []
alens = []
for iquestion in iquestions:
    question_string = iquestion['question'][0]['string']
    sparql_query = iquestion['query']['sparql']
    answer = iquestion['answers']
    qlens.append(len(iquestion['question']))
    alens.append(len(answer))
    question_strings.append(question_string)
    spqrql_queries.append(sparql_query)
    answers.append(answer)


In [8]:
#parse sparql
#import sys
#!{sys.executable} -m pip install rdflib
import rdflib
from rdflib.plugins.sparql.parser import parseQuery
from pprint import pprint

parsed_queries = []
for query in spqrql_queries:
    try:
        query = query.replace('COUNT(?uri)', '(COUNT(*) AS ?callret)') #make the count queries compatible with rdflib 
        q = parseQuery(query)[1]
        parsed_queries.append(q)
    except:
         print(query)

In [9]:
#example of parse query
q = parseQuery('SELECT DISTINCT (COUNT(*) AS ?callret) WHERE { ?x <http://dbpedia.org/ontology/builder> <http://dbpedia.org/resource/PCL_Construction> . ?x <http://dbpedia.org/ontology/tenant> ?uri  . }')[1]

q['where']['part'][0]['triples'][0][1]['part'][0]['part'][0]['part'] #extremely ugly

rdflib.term.URIRef('http://dbpedia.org/ontology/builder')

In [10]:
#how often is the DISTINCT modifier present in the dataset. 
from collections import Counter

modifiers = [q.modifier for q in parsed_queries]
Counter(modifiers)

Counter({'DISTINCT': 4632, None: 368})

In [11]:
#print the query types in the dataset. 
query_types = [q.name for q in parsed_queries]
Counter(query_types)

Counter({'SelectQuery': 4632, 'AskQuery': 368})

In [12]:
#Here, we look at how many triples there are in the SPARQL queries, where a triple is e.g. ?x rel subject
triplets = [q['where']['part'][0]['triples'] for q in parsed_queries]
Counter([len(triple) for triple in triplets])

Counter({2: 2103, 1: 1368, 3: 1529})

In [13]:
projection = [q.get('projection', None) for q in parsed_queries]
pprint(projection[15])
pprint(projection[16])
pprint(projection[0])

[{'evar': rdflib.term.Variable('callret'),
  'expr': {'expr': {'expr': {'expr': {'expr': {'expr': {'distinct': ([], {}),
                                                        'vars': '*'}}}}}}}]
'projection'
[vars_{'var': rdflib.term.Variable('uri')}]


In [14]:
parsed_triplets = []
for triples in triplets:
    parsed_triples = []
    for triple in triples:
        rel = triple[1]['part'][0]['part'][0]['part']
        parsed_triples.append((triple[0], rel, triple[2]))
    parsed_triplets.append(parsed_triples)

In [15]:
parsed_triplets[144]

[(rdflib.term.Variable('x'),
  rdflib.term.URIRef('http://dbpedia.org/ontology/ingredient'),
  rdflib.term.URIRef('http://dbpedia.org/resource/Shallot')),
 (rdflib.term.Variable('x'),
  rdflib.term.URIRef('http://dbpedia.org/ontology/country'),
  rdflib.term.Variable('uri')),
 (rdflib.term.Variable('uri'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://dbpedia.org/ontology/Country'))]

In [16]:
parsed_triplets[144][0][1].n3()

'<http://dbpedia.org/ontology/ingredient>'

In [17]:
#get all the entities present in the SQA dataset - use them later as seeds for the random walks
entities = set()
relations = set()

for triples in parsed_triplets:
    for e1, r, e2 in triples:
        #add all entities which are not variables
        if not e1.n3() == '?uri' and not e1.n3() == '´?x':
            entities.add(e1.n3())
        if not e2.n3() == '?uri' and not e2.n3() == '´?x':
            entities.add(e2.n3())
        relations.add(r.n3())
print(len(entities), len(relations))

4156 597


In [25]:
import random

random.sample(list(entities), k=10)

['<http://dbpedia.org/resource/Gabriel_París_Gordillo>',
 '<http://dbpedia.org/resource/Edward_Douglass_White>',
 '<http://dbpedia.org/resource/Kimberly_Stewart>',
 '<http://dbpedia.org/resource/Salsette_Island>',
 '<http://dbpedia.org/resource/Companion_(Doctor_Who)>',
 '<http://dbpedia.org/resource/Yabancı_Damat>',
 '<http://dbpedia.org/resource/Texas>',
 '<http://dbpedia.org/resource/Henry_Howard,_1st_Earl_of_Northampton>',
 '<http://dbpedia.org/resource/Cobra_(missile)>',
 '<http://dbpedia.org/resource/Marika_Gombitová>']

In [14]:
import random

random.sample(list(relations), k=10)

['<http://dbpedia.org/ontology/wineRegion>',
 '<http://dbpedia.org/ontology/sourceMountain>',
 '<http://dbpedia.org/ontology/foundationPlace>',
 '<http://dbpedia.org/ontology/parentMountainPeak>',
 '<http://dbpedia.org/ontology/denomination>',
 '<http://dbpedia.org/property/headCoach>',
 '<http://dbpedia.org/property/nationalteam>',
 '<http://dbpedia.org/ontology/nonFictionSubject>',
 '<http://dbpedia.org/property/party>',
 '<http://dbpedia.org/ontology/riverMouth>']

# Andrea

In [15]:
#store the list of entities in the SQA datafolder
ofile = open('sqa_entities.txt', 'wt', encoding='utf-8')
ofile.writelines([x + '\n' for x in list(entities)])
ofile.close()

In [16]:
#store the list of questions in the SQA datafolder
ofile = open('sqa_questions.txt', 'wt', encoding='utf-8')
ofile.writelines([x + '\n' for x in question_strings])
ofile.close()

In [17]:
#store the list of SPARQL queries in the SQA datafolder
ofile = open('sqa_sparql.txt', 'wt', encoding='utf-8')
ofile.writelines([x[0][1].n3() + '\n' for x in parsed_triplets])
ofile.close()

In [27]:
#extract the questions, the SPARQL queries and the answers, including index
question_strings = []
spqrql_queries = []
answers = []

qlens = []
alens = []

ofile = open('sqa_questions_index.txt', 'wt', encoding='utf-8')
ofile2 = open('sqa_sparql_index.txt', 'wt', encoding='utf-8')
for i, iquestion in enumerate(iquestions):
    question_string = iquestion['question'][0]['string']
    sparql_query = iquestion['query']['sparql']
    answer = iquestion['answers']
    qlens.append(len(iquestion['question']))
    alens.append(len(answer))
    question_strings.append(question_string)
    spqrql_queries.append(sparql_query)
    answers.append(answer)
    ofile.writelines(str(i) + ',' + question_string + '\n' )
    ofile2.writelines(str(i) + ',' + sparql_query + '\n' )
    
ofile.close()
ofile2.close()

In [18]:
#store the list of SPARQL queries with index
ofile = open('sqa_sparql_parsed.txt', 'wt', encoding='utf-8')
for i, x in enumerate(parsed_triplets):
    ofile.writelines(str(i) + ',' + str(x) + '\n')
    
ofile.close()

In [29]:
#get all the entities present in the SQA dataset - use them later as seeds for the random walks
entities = set()
relations = set()

ofile = open('sqa_entities_index.txt', 'wt', encoding='utf-8')
ofile2 = open('sqa_relations_index.txt', 'wt', encoding='utf-8')

for i, triples in enumerate(parsed_triplets):
    for e1, r, e2 in triples:
        #add all entities which are not variables
        if not e1.n3() == '?uri' and not e1.n3() == '?x':
            entities.add(e1.n3())
            ofile.writelines(str(i) + ',' + e1.n3() + '\n')
        if not e2.n3() == '?uri' and not e2.n3() == '?x':
            entities.add(e2.n3())
            ofile.writelines(str(i) + ',' + e2.n3() + '\n')
        relations.add(r.n3())
        ofile2.writelines(str(i) + ',' + r.n3() + '\n')
print(len(entities), len(relations))
ofile.close()
ofile2.close()

4155 597


In [21]:
# match entities 
ifile =  open('sqa_questions_index.txt', 'rt', encoding='utf-8')
ifile2 =  open('sqa_entities_index.txt', 'rt', encoding='utf-8')

#ofile =  open('sqa_entities_matched.txt', 'rt', encoding='utf-8')
#ofile2 =  open('sqa_entities_notmatched.txt', 'rt', encoding='utf-8')

matched = []
notmatched = []
for line in ifile:
    #print("line", line)
    idx, question = line.split(',', 1)
    #print(question)
    for line2 in ifile2:
        #print("entity", line2)
        idx2, entity = line2.split(',', 1)
        if idx2 == idx:
            #print(entity)
            if entity in question:
                matched.append(entity)
            else:
                notmatched.append(entity)
        else:
            break
                
print(len(matched), len(notmatched))

0 3547


In [22]:
ifile =  open('sqa_questions_index.txt', 'rt', encoding='utf-8')
ifile2 =  open('sqa_entities_index.txt', 'rt', encoding='utf-8')

matched = []
notmatched = []

line = '0,Which comic characters are painted by Bill Finger?'
idx, question = line.split(',', 1)

line2 = '0,<http://dbpedia.org/resource/Bill_Finger>'
idx2, entity1 = line2.split(',',1)
line3 = '0,<http://dbpedia.org/ontology/ComicsCharacter>'
idx3, entity2 = line3.split(',',1)

entities = [entity1, entity2]
print(question)
print(entity1)
print(entity2)

for entity in entities:
    if entity in question:
        matched.append(entity)
    else:
        notmatched.append(entity)
        
import re

_, entity_parsed = entity1.rsplit('/',1)
print(entity_parsed.replace('>',''))
reg = re.compile(r'<http://dbpedia.org/\w+/(\w+)>')
print(reg.findall(entity2))
    
print(len(matched), len(notmatched))

Which comic characters are painted by Bill Finger?
<http://dbpedia.org/resource/Bill_Finger>
<http://dbpedia.org/ontology/ComicsCharacter>
Bill_Finger
['ComicsCharacter']
0 2


In [43]:
************************************************************************************************************
************************************************************************************************************
************************************************************************************************************


SyntaxError: invalid syntax (<ipython-input-43-cee125ecf0de>, line 1)

In [23]:
matched = []
notmatched = []

import re

with open('sqa_questions_index.txt', 'r', encoding='utf-8') as ifile:
    for line in ifile:
        line = line.replace('\n', '')
        idx, question = line.split(',', 1)
        question = question.replace('?', '')
        #print(line)
        with open('sqa_entities_index.txt', 'rt', encoding='utf-8') as ifile2:
            for line2 in ifile2:
                line2 = line2.replace('\n', '')
                idx2, entity = line2.split(',',1)
                #print(line2)
                #print(idx2)
                if idx2 == idx:
                    #print(line2)
                    if entity in question:
                        matched.append(entity)
                    else:
                        notmatched.append(entity)

        ifile2.close()

        
print(len(matched), len(notmatched))

0 8546


In [ ]:
# we have 8546 entities in total, with duplications.
# we check if each entity for each question can be identified in the question.
# without any string modification, any URI can't be found in the questions.

In [1]:
matched = []
notmatched = []

with open('sqa_questions_index.txt', 'r', encoding='utf-8') as ifile:
    for line in ifile:
        line = line.replace('\n', '')
        idx, question = line.split(',', 1)
        question = question.replace('?', '')
        #print(line)
        with open('sqa_entities_index.txt', 'rt', encoding='utf-8') as ifile2:
            for line2 in ifile2:
                line2 = line2.replace('\n', '')
                idx2, entity = line2.split(',',1)
                #print(line2)
                #print(idx2)
                if idx2 == idx:
                    #print(line2)
                    _, entity = entity.rsplit('/',1)
                    entity_parsed = entity.replace('>','')
                    #reg = re.compile(r'<http://dbpedia.org/\w+/([\(\w\),.\']+)>')
                    #entity_parsed = reg.findall(entity)
                    ##print(entity_parsed)
                    #entity_parsed = entity_parsed[0]
                    if entity_parsed in question:
                        matched.append(entity_parsed)
                    else:
                        notmatched.append(entity_parsed)

        ifile2.close()

print(matched[0])        
print(len(matched), len(notmatched))

Gestapo
849 7697


In [24]:
# taking the last element from the uri, we match 849 entities (9.93%)
#print(matched)

In [2]:
matched = []
notmatched = []

with open('sqa_questions_index.txt', 'r', encoding='utf-8') as ifile:
    for line in ifile:
        line = line.replace('\n', '')
        idx, question = line.split(',', 1)
        question = question.replace('?', '')
        question_lower = question.lower()
        #print(line)
        with open('sqa_entities_index.txt', 'rt', encoding='utf-8') as ifile2:
            for line2 in ifile2:
                line2 = line2.replace('\n', '')
                idx2, entity = line2.split(',',1)
                #print(line2)
                #print(idx2)
                if idx2 == idx:
                    #print(line2)
                    _, entity = entity.rsplit('/',1)
                    entity_parsed = entity.replace('>','')
                    entity_parsed_lower = entity_parsed.lower()
                    if entity_parsed in question or entity_parsed_lower in question or entity_parsed in question_lower or entity_parsed_lower in question_lower:
                        matched.append(entity_parsed)
                    else:
                        notmatched.append(entity_parsed)

        ifile2.close()

print(matched[0])        
print(len(matched), len(notmatched))

Gestapo
1762 6784


In [ ]:
# taking the last element from the uri and considering the original word(s) and with lowercase, we match 1762 entities (20.62%)

In [25]:
random.sample(list(notmatched), k=10)

['Olivier_Boulay',
 'Doctor_Who:_The_Mazes_of_Time',
 'Cahora_Bassa',
 'Edwin_F._Hunter',
 'North_Carolina_Highway_280',
 'Steven_Palazzo',
 'Public_Transport_Authority_(Western_Australia)',
 'Pontiac_Building',
 'George_Karrys',
 'Duck_family_(Disney)']

In [81]:
matched = []
notmatched = []
question_notmatched = []

with open('sqa_questions_index.txt', 'r', encoding='utf-8') as ifile:
    for line in ifile:
        line = line.replace('\n', '')
        idx, question = line.split(',', 1)
        question = question.replace('?', '')
        question_lower = question.lower()
        #print(line)
        with open('sqa_entities_index.txt', 'rt', encoding='utf-8') as ifile2:
            for line2 in ifile2:
                line2 = line2.replace('\n', '')
                idx2, entity = line2.split(',',1)
                #print(line2)
                #print(idx2)
                if idx2 == idx:
                    #print(line2)
                    _, entity = entity.rsplit('/',1)
                    entity_parsed = entity.replace('>','')
                    entity_parsed = entity_parsed.replace('_', ' ')
                    entity_parsed_lower = entity_parsed.lower()
                    if entity_parsed in question or entity_parsed_lower in question or entity_parsed in question_lower or entity_parsed_lower in question_lower:
                        matched.append(entity_parsed)
                    else:
                        notmatched.append(entity_parsed)
                        question_notmatched.append(question)

        ifile2.close()

        
print(len(matched), len(notmatched))

5778 2768


In [ ]:
# replacing hyphen with blank space, we match 5778 entities (67.61%)

In [28]:
random.sample(list(notmatched), k=10)

['TelevisionShow',
 'New York City FC',
 'Henry McDaniel (racehorse trainer)',
 'Scientology',
 'ScreenWriter',
 'TelevisionShow',
 'Film',
 'Whitey Wistert',
 'Craig Van Tilbury',
 'VideoGame']

In [29]:
print(question_notmatched[0:10])
print(notmatched[0:10])

['Which comic characters are painted by Bill Finger', 'Which party has come in power in Mumbai North', 'Which royalty was married to ptolemy XIII Theos Philopator and had mother named Cleopatra V ', 'Which company which assembles its cars in Broadmeadows, Victoria', 'Which person designed the cars which has been designed by ASC Creative Services', 'What tv series can be said to be related to the sarah jane adventure and dr who confidential', 'What tv series can be said to be related to the sarah jane adventure and dr who confidential', 'How many shows does HBO have', 'Is Nikolai Morozov the former coach of Stanislav Morozov', 'What is the common genere of Abhijit Kunte and Kasparov ']
['ComicsCharacter', 'Mumbai North (Lok Sabha constituency)', 'Cleopatra V of Egypt', 'Automobile', 'Automobile', 'The Sarah Jane Adventures', 'Doctor Who Confidential', 'TelevisionShow', 'Nikolai Morozov (figure skater)', 'Virtual Kasparov']


In [82]:
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = stopwords.words('english')
from nltk.stem.snowball import SnowballStemmer
STEMMER = SnowballStemmer("english")

a = 'ComicsCharacter'
b = 'Adventures'

bb = STEMMER.stem(b)
print(bb)

a1 = ''
a2 = ''
idx = 0
for i in range(0,len(a)):
    if i>0 and a[i].isupper() == True:
        idx = i

a1 = a[:idx]
a2 = a[idx:]
    
print(a1)
print(a2)

adventur
Comics
Character


In [83]:
#matched = []
#notmatched = []
#question_notmatched = []

matched2 = []
notmatched2 = []
question_notmatched2 = []

a1 = ''
a2 = ''
a3 = ''
idx2 = 0
idx = 0

print(len(notmatched))      
print(len(question_notmatched))


for entity in notmatched:
    #idx = notmatched.index(entity)
    #print(idx)
    #print(entity)
    if sum(1 for x in entity if x.isspace())>0:
        notmatched2.append(entity)
        question_notmatched2.append(question_notmatched[idx])
        #print(entity)
        #print(idx)
    else:
        if sum(1 for x in entity if x.isupper()) > 1:
            index = []
            for i in range(0,len(entity)):
                if i>0 and entity[i].isupper() == True:                 
                    index.append(i)
                                                          
            question_notmatched_lower = question_notmatched[idx].lower()
            if len(index) == 1:
                idx2 = index[0]
                a1 = entity[:idx2].lower()
                a2 = entity[idx2:].lower()
                if a1 in question_notmatched[idx] or a2 in question_notmatched[idx] or a1 in question_notmatched_lower or a2 in question_notmatched_lower:
                    matched2.append(entity)
                else:
                    notmatched2.append(entity)
                    question_notmatched2.append(question_notmatched[idx]) 
            elif len(index) == 2:
                if 'TV' in entity:
                    idx2 = entity.index('TV')
                    a1 = entity[:idx2].lower()
                    a2 = entity[idx2:].lower()
                    if a1 in question_notmatched[idx] or a2 in question_notmatched[idx] or a1 in question_notmatched_lower or a2 in question_notmatched_lower:
                        matched2.append(entity)
                    else:
                        notmatched2.append(entity)
                        question_notmatched2.append(question_notmatched[idx]) 
                    
                else:  
                    idx2 = index[0]
                    idx3 = index[1]
                    #print(entity)
                    
                    if '-' in entity:
                        entity = entity.replace('-', '')                 
                       
                    a1 = entity[:idx2].lower()
                    a2 = entity[idx2:idx3].lower()
                    a3 = entity[idx3:].lower()
                    #print(a1, a2, a3)
                    
                    # 'non' has to appear in the question, and also one of the other words
                    if '-' in entity:
                        if a1 in question_notmatched[idx] or a1 in question_notmatched_lower \
                        and (a2 in question_notmatched[idx] or a3 in question_notmatched[idx] \
                            or a2 in question_notmatched_lower or a3 in question_notmatched_lower):
                            matched2.append(entity)
                        else:
                            notmatched2.append(entity)
                            question_notmatched2.append(question_notmatched[idx])  
                            
                    else:
                        if a1 in question_notmatched[idx] or a2 in question_notmatched[idx] or a3 in question_notmatched[idx] \
                            or a1 in question_notmatched_lower or a2 in question_notmatched_lower or a3 in question_notmatched_lower:
                            matched2.append(entity)
                        else:
                            notmatched2.append(entity)
                            question_notmatched2.append(question_notmatched[idx])     
                    
            else:
                # case: abreviation of a radio channel. For now we will exclude it, goes directly to non-matched entities group
                #print(entity)
                notmatched2.append(entity)
                question_notmatched2.append(question_notmatched[idx])     
                
        elif sum(1 for x in entity if x.isupper()) == 1:
            #print(entity)
            notmatched2.append(entity)
            question_notmatched2.append(question_notmatched[idx])    
        else:
            print(entity)
               
        
    idx +=1


#for ent in notmatched:
 #   if ent not in matched2 and ent not in notmatched2:
  #    print(ent)

print(len(matched2), len(notmatched2))

2768
2768
653 2115


In [ ]:
# entities like 'ComicsCharacter' splitted in 'comics' and 'character', and try to map in the question
# we check just in the non matched till now

#we match 653 more entities (5778 + 653 --> 75.25%)

In [32]:
print(question_notmatched2[0:10])
print(notmatched2[0:10])

['Which party has come in power in Mumbai North', 'Which royalty was married to ptolemy XIII Theos Philopator and had mother named Cleopatra V ', 'Which company which assembles its cars in Broadmeadows, Victoria', 'Which person designed the cars which has been designed by ASC Creative Services', 'What tv series can be said to be related to the sarah jane adventure and dr who confidential', 'What tv series can be said to be related to the sarah jane adventure and dr who confidential', 'Is Nikolai Morozov the former coach of Stanislav Morozov', 'What is the common genere of Abhijit Kunte and Kasparov ', 'Which share holder of outlook is also the content license of the MSX Basics ', 'Which source country of SamurAbsheron channel is the birth place of Anar Salmanov ']
['Mumbai North (Lok Sabha constituency)', 'Cleopatra V of Egypt', 'Automobile', 'Automobile', 'The Sarah Jane Adventures', 'Doctor Who Confidential', 'Nikolai Morozov (figure skater)', 'Virtual Kasparov', 'Outlook on the web'

In [53]:
# remove parenthesis, of ..., on ...
entities = []

for entity in notmatched2:
    idx = notmatched2.index(entity)
    if '(' in entity and ')' in entity:
        entity, _ = entity.split('(', 1)
        
    if 'of' in entity:
        entity, _ = entity.split('of', 1)
    
    if 'on' in entity:
        entity, _ = entity.split('on', 1)
    
    entities.append(entity.strip())
    
#print(entities)


matched3 = []
notmatched3 = []
question_notmatched3 = []

print(len(question_notmatched2))
print(len(notmatched2))
for idx, question in enumerate(question_notmatched2):
    #print(question)
    question_lower = question.lower()
    entity = entities[idx]
    entity_parsed_lower = entity.lower()
    if entity in question or entity_parsed_lower in question or entity in question_lower or entity_parsed_lower in question_lower:
        matched3.append(entity)
    else:
        notmatched3.append(entity)
        question_notmatched3.append(question)


print(len(matched3), len(notmatched3))

2115
2115
574 1541


In [ ]:
# simplifying the entities (removing the extra info in parenthesis and info after 'of', 'on'), we matched 574 more entities
# (6431 + 574 --> 81.97%)

In [34]:
print(question_notmatched3[0:10])
print(notmatched3[0:10])

['Which company which assembles its cars in Broadmeadows, Victoria', 'Which person designed the cars which has been designed by ASC Creative Services', 'What tv series can be said to be related to the sarah jane adventure and dr who confidential', 'What tv series can be said to be related to the sarah jane adventure and dr who confidential', 'What is the common genere of Abhijit Kunte and Kasparov ', 'Which source country of SamurAbsheron channel is the birth place of Anar Salmanov ', 'Where was the battle fought where 2nd Foreign Infantry Regiment participated  ', 'Did Timm Gunn guest in Sunrise (HIMYM)', 'which politician is currently ruling over Rishkiesh', 'Is north america one of the destinations of airtours airways']
['Automobile', 'Automobile', 'The Sarah Jane Adventures', 'Doctor Who C', 'Virtual Kasparov', 'Samur–Absher', 'MilitaryC', 'Tim Gunn', 'Rishikesh', 'Airtours Internati']


In [84]:
from nltk.stem.snowball import SnowballStemmer
STEMMER = SnowballStemmer("english")

matched4 = []
notmatched4 = []
question_notmatched4 = []

print(len(question_notmatched3))
print(len(notmatched3))
for idx, question in enumerate(question_notmatched3):
    #print(question)
    question = question.replace('?', '')
    question_lower = question.lower()
    entity = notmatched3[idx]
    entity_parsed_lower = entity.lower()    
    entity_stem = STEMMER.stem(entity_parsed_lower)
    
    if entity_stem in question or entity_stem in question_lower:
        matched4.append(entity)
    else:
        notmatched4.append(entity)
        question_notmatched4.append(question)


print(len(matched4), len(notmatched4))

1541
1541
117 1424


In [ ]:
# simplifying the entities (using STEMMER, to consider the root of the word), we matched 117 more entities
# (7005 + 117 --> 83.34%)

In [36]:
print(question_notmatched4[0:10])
print(notmatched4[0:10])

['Which company which assembles its cars in Broadmeadows, Victoria', 'Which person designed the cars which has been designed by ASC Creative Services', 'What tv series can be said to be related to the sarah jane adventure and dr who confidential', 'What is the common genere of Abhijit Kunte and Kasparov ', 'Which source country of SamurAbsheron channel is the birth place of Anar Salmanov ', 'Where was the battle fought where 2nd Foreign Infantry Regiment participated  ', 'Did Timm Gunn guest in Sunrise (HIMYM)', 'which politician is currently ruling over Rishkiesh', 'Is north america one of the destinations of airtours airways', 'Who is the common manager of Aston Villa 2000-02 season and aldo the manager of Middlesbrough F.C. season 2009-10']
['Automobile', 'Automobile', 'Doctor Who C', 'Virtual Kasparov', 'Samur–Absher', 'MilitaryC', 'Tim Gunn', 'Rishikesh', 'Airtours Internati', '2000–01 Ast']


In [ ]:
# if a entity is compound for 2 words or more, try to match any of them in the question
# 'virtual kasparov' --> kasparov will be matched

In [85]:
matched5 = []
notmatched5 = []
question_notmatched5 = []
entities = []

print(len(question_notmatched4))
print(len(notmatched4))
for idx, question in enumerate(question_notmatched4):
    #print(question)
    question = question.replace('?', '')
    question_lower = question.lower()
    entity = notmatched4[idx]
    entity_parsed_lower = entity.lower()    
    for e in entity_parsed_lower.split(' '):
        entities.append(e)
    
    
    if any (ent in question or ent in question_lower for ent in entities):
        matched5.append(entity)
    else:
        notmatched5.append(entity)
        question_notmatched5.append(question)

        
    entities = []

print(len(matched5), len(notmatched5))

1424
1424
816 608


In [ ]:
# splitting compound entities and trying to match any of the words, we matched 820 more entities
# (7122 + 816 --> 92.88%)

In [38]:
print(question_notmatched5[0:10])
print(notmatched5[0:10])

['Which company which assembles its cars in Broadmeadows, Victoria', 'Which person designed the cars which has been designed by ASC Creative Services', 'Which source country of SamurAbsheron channel is the birth place of Anar Salmanov ', 'Where was the battle fought where 2nd Foreign Infantry Regiment participated  ', 'which politician is currently ruling over Rishkiesh', 'Which labels signs up progressive rock bands', 'How many people have headed organizations headquartered at Niederkirchnerstrae', 'Which tenants of the Master Card center is also th draft team of Sam Carrrick ', 'Whose provides the service of  Outlook and also products  such as Visual Studio', 'Whose provides the service of  Outlook and also products  such as Visual Studio']
['Automobile', 'Automobile', 'Samur–Absher', 'MilitaryC', 'Rishikesh', 'MusicalArtist', 'Niederkirchnerstraße', 'MasterCard Centre', 'Outlook.com', 'Micros']


In [86]:
ofile = open('question_notmatched5.txt', 'wt', encoding='utf-8')
ofile2 = open('entities_notmatched5.txt', 'wt', encoding='utf-8')

for i, question in enumerate(question_notmatched5):
    print(question)
    print(notmatched5[i])
    ofile.writelines(str(i) + ',' + str(question) + '\n')
    ofile2.writelines(str(i) + ',' + str(notmatched5[i]) + '\n')

ofile.close()
ofile2.close()

Which company which assembles its cars in Broadmeadows, Victoria
Automobile
Which person designed the cars which has been designed by ASC Creative Services
Automobile
Which source country of SamurAbsheron channel is the birth place of Anar Salmanov 
Samur–Absher
Where was the battle fought where 2nd Foreign Infantry Regiment participated  
MilitaryC
which politician is currently ruling over Rishkiesh
Rishikesh
Which labels signs up progressive rock bands
MusicalArtist
How many people have headed organizations headquartered at Niederkirchnerstrae
Niederkirchnerstraße
Which tenants of the Master Card center is also th draft team of Sam Carrrick 
MasterCard Centre
Whose provides the service of  Outlook and also products  such as Visual Studio
Outlook.com
Whose provides the service of  Outlook and also products  such as Visual Studio
Micros
list  the producer of the movies in which Rosemarie DeWitt in the star cast
Film
What is the city whose mayor is Giorgos Kaminis
AdministrativeRegi
How

In [5]:
q = 'which politician is currently ruling over rishkiesh?'
a = 'rishikesh'

import difflib
#import sys
#!{sys.executable} -m pip install python-levenshtein


def LD(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    if s[-1] == t[-1]:
        cost = 0
    else:
        cost = 1
       
    res = min([LD(s[:-1], t)+1,
               LD(s, t[:-1])+1, 
               LD(s[:-1], t[:-1]) + cost])
    return res
#print(LD("Python", "Peithen"))


for w in q.replace('?', '').split(' '):
    print(w)
    print(LD(a, w))

which
6
politician
9
is
7
currently
9
ruling
7
over
8
rishkiesh
2


In [46]:
matched6 = []
notmatched6 = []
question_notmatched6 = []


for idx, question in enumerate(question_notmatched5):
    question = question.replace('?', '')
    question = question.lower()
    print(question)
    entity = notmatched5[idx]
    entity_parsed_lower = entity.lower() 
    #for w in question.split(' '):
        #print(w)
        
    try:
        if any (LD(entity_parsed_lower, w) < 3 for w in question.split(' ')):
            print(a)
            print(w)
            matched6.append(entity)
        else:
            print("NO")
            notmatched6.append(entity)
            question_notmatched6.append(question)

            #if LD(entity_parsed_lower, w) < 3:
            #  print(a)
             #   print(w)
             #   matched6.append(entity)
            #else:
            #    notmatched6.append(entity)
             #   question_notmatched6.append(question)
    except:
        print(idx)
        continue

print(len(matched6), len(notmatched))

NameError: name 'question_notmatched5' is not defined

In [ ]:
matched6 = []
notmatched6 = []
question_notmatched6 = []
distances = []

def LD(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    if s[-1] == t[-1]:
        cost = 0
    else:
        cost = 1
       
    res = min([LD(s[:-1], t)+1,
               LD(s, t[:-1])+1, 
               LD(s[:-1], t[:-1]) + cost])
    return res

with open('question_notmatched5.txt', 'r', encoding='utf-8') as ifile:
    for line in ifile:
        idx, question = line.split(',', 1)
        print(question)
        with open('entities_notmatched5.txt', 'rt', encoding='utf-8') as ifile2:
            for line2 in ifile2:
                idx2, entity = line2.split(',',1)
                if idx2 == idx:
                    #print(entity)
                    question = question.replace('?', '')
                    question = str(question.lower())
                    entity_parsed_lower = str(entity.lower())
                    print(question)
                    print(entity_parsed_lower)
                    
                    print("TRYING")
                    for w in question.split(' '):
                        try:
                            w = w.replace(',', '')
                            print(w)
                            distances.append(LD(entity_parsed_lower, w))
                            print(distances)
                        except:
                            continue
                            
                    if any(d < 3 for d in distances):
                        print("YES")
                        matched6.append(entity)
                                
                    else:
                        print("NO")
                        notmatched6.append(entity)
                        question_notmatched6.append(question)

                            #if LD(entity_parsed_lower, w) < 3:
                            #  print(a)
                             #   print(w)
                             #   matched6.append(entity)
                            #else:
                            #    notmatched6.append(entity)
                             #   question_notmatched6.append(question)

                distances = []


Which company which assembles its cars in Broadmeadows, Victoria

which company which assembles its cars in broadmeadows, victoria

automobile

TRYING
which
[10]
company
[10, 9]
which
[10, 9, 10]
assembles
[10, 9, 10, 6]
its
[10, 9, 10, 6, 10]
cars
[10, 9, 10, 6, 10, 11]
in
[10, 9, 10, 6, 10, 11, 10]
broadmeadows
[10, 9, 10, 6, 10, 11, 10, 11]
victoria

[10, 9, 10, 6, 10, 11, 10, 11, 8]
NO
Which person designed the cars which has been designed by ASC Creative Services

which person designed the cars which has been designed by asc creative services

automobile

TRYING
which
[10]
person
[10, 10]
designed
[10, 10, 10]
the
[10, 10, 10, 9]
cars
[10, 10, 10, 9, 11]
which
[10, 10, 10, 9, 11, 10]
has
[10, 10, 10, 9, 11, 10, 11]
been
[10, 10, 10, 9, 11, 10, 11, 9]
designed
[10, 10, 10, 9, 11, 10, 11, 9, 10]
by
[10, 10, 10, 9, 11, 10, 11, 9, 10, 10]
asc
[10, 10, 10, 9, 11, 10, 11, 9, 10, 10, 10]
creative
[10, 10, 10, 9, 11, 10, 11, 9, 10, 10, 10, 9]
services

[10, 10, 10, 9, 11, 10, 11, 9, 10, 1

In [3]:
def iterative_levenshtein(s, t, costs=(1, 1, 1)):
    """ 
        iterative_levenshtein(s, t) -> ldist
        ldist is the Levenshtein distance between the strings 
        s and t.
        For all i and j, dist[i,j] will contain the Levenshtein 
        distance between the first i characters of s and the 
        first j characters of t
        
        costs: a tuple or a list with three integers (d, i, s)
               where d defines the costs for a deletion
                     i defines the costs for an insertion and
                     s defines the costs for a substitution
    """
    rows = len(s)+1
    cols = len(t)+1
    deletes, inserts, substitutes = costs
    
    dist = [[0 for x in range(cols)] for x in range(rows)]
    # source prefixes can be transformed into empty strings 
    # by deletions:
    for row in range(1, rows):
        dist[row][0] = row * deletes
    # target prefixes can be created from an empty source string
    # by inserting the characters
    for col in range(1, cols):
        dist[0][col] = col * inserts
        
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0
            else:
                cost = substitutes
            dist[row][col] = min(dist[row-1][col] + deletes,
                                 dist[row][col-1] + inserts,
                                 dist[row-1][col-1] + cost) # substitution
    for r in range(rows):
        print(dist[r])
    
 
    return dist[row][col]




q = 'which politician is currently ruling over rishkiesh?'
a = 'rishikesh'

for w in q.replace('?', '').split(' '):
    print(w)
    print(iterative_levenshtein(a, w))


which
[0, 1, 2, 3, 4, 5]
[1, 1, 2, 3, 4, 5]
[2, 2, 2, 2, 3, 4]
[3, 3, 3, 3, 3, 4]
[4, 4, 3, 4, 4, 3]
[5, 5, 4, 3, 4, 4]
[6, 6, 5, 4, 4, 5]
[7, 7, 6, 5, 5, 5]
[8, 8, 7, 6, 6, 6]
[9, 9, 8, 7, 7, 6]
6
politician
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2, 2, 2, 3, 3, 4, 5, 6, 7, 8, 9]
[3, 3, 3, 3, 4, 4, 5, 6, 7, 8, 9]
[4, 4, 4, 4, 4, 5, 5, 6, 7, 8, 9]
[5, 5, 5, 5, 4, 5, 5, 6, 6, 7, 8]
[6, 6, 6, 6, 5, 5, 6, 6, 7, 7, 8]
[7, 7, 7, 7, 6, 6, 6, 7, 7, 8, 8]
[8, 8, 8, 8, 7, 7, 7, 7, 8, 8, 9]
[9, 9, 9, 9, 8, 8, 8, 8, 8, 9, 9]
9
is
[0, 1, 2]
[1, 1, 2]
[2, 1, 2]
[3, 2, 1]
[4, 3, 2]
[5, 4, 3]
[6, 5, 4]
[7, 6, 5]
[8, 7, 6]
[9, 8, 7]
7
currently
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 1, 2, 2, 3, 4, 5, 6, 7, 8]
[2, 2, 2, 3, 3, 4, 5, 6, 7, 8]
[3, 3, 3, 3, 4, 4, 5, 6, 7, 8]
[4, 4, 4, 4, 4, 5, 5, 6, 7, 8]
[5, 5, 5, 5, 5, 5, 6, 6, 7, 8]
[6, 6, 6, 6, 6, 6, 6, 7, 7, 8]
[7, 7, 7, 7, 7, 6, 7, 7, 8, 8]
[8, 8, 8, 8, 8, 7, 7, 8, 8, 9]
[9, 9, 9, 9, 9, 8, 8, 8, 9, 9]
9
ruling
[0, 1, 2, 3, 

In [4]:
matched6 = []
notmatched6 = []
question_notmatched6 = []
distances = []

def LD(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    if s[-1] == t[-1]:
        cost = 0
    else:
        cost = 1
       
    res = min([LD(s[:-1], t)+1,
               LD(s, t[:-1])+1, 
               LD(s[:-1], t[:-1]) + cost])
    return res

with open('question_notmatched5.txt', 'r', encoding='utf-8') as ifile:
    for line in ifile:
        idx, question = line.split(',', 1)
        print(question)
        with open('entities_notmatched5.txt', 'rt', encoding='utf-8') as ifile2:
            for line2 in ifile2:
                idx2, entity = line2.split(',',1)
                if idx2 == idx:
                    #print(entity)
                    question = question.replace('?', '')
                    question = str(question.lower())
                    entity_parsed_lower = str(entity.lower())
                    print(question)
                    print(entity_parsed_lower)
                    
                    print("TRYING")
                    for w in question.split(' '):
                        try:
                            w = w.replace(',', '')
                            #print(w)
                            distances.append(iterative_levenshtein(entity_parsed_lower, w))
                            #print(distances)
                        except:
                            continue
                            
                    if any(d < 3 for d in distances):
                        print("YES")
                        matched6.append(entity)
                                
                    else:
                        print("NO")
                        notmatched6.append(entity)
                        question_notmatched6.append(question)

                            #if LD(entity_parsed_lower, w) < 3:
                            #  print(a)
                             #   print(w)
                             #   matched6.append(entity)
                            #else:
                            #    notmatched6.append(entity)
                             #   question_notmatched6.append(question)

                distances = []


Which company which assembles its cars in Broadmeadows, Victoria

which company which assembles its cars in broadmeadows, victoria

automobile

TRYING
[0, 1, 2, 3, 4, 5]
[1, 1, 2, 3, 4, 5]
[2, 2, 2, 3, 4, 5]
[3, 3, 3, 3, 4, 5]
[4, 4, 4, 4, 4, 5]
[5, 5, 5, 5, 5, 5]
[6, 6, 6, 6, 6, 6]
[7, 7, 7, 7, 7, 7]
[8, 8, 8, 7, 8, 8]
[9, 9, 9, 8, 8, 9]
[10, 10, 10, 9, 9, 9]
[11, 11, 11, 10, 10, 10]
[0, 1, 2, 3, 4, 5, 6, 7]
[1, 1, 2, 3, 4, 4, 5, 6]
[2, 2, 2, 3, 4, 5, 5, 6]
[3, 3, 3, 3, 4, 5, 6, 6]
[4, 4, 3, 4, 4, 5, 6, 7]
[5, 5, 4, 3, 4, 5, 6, 7]
[6, 6, 5, 4, 4, 5, 6, 7]
[7, 7, 6, 5, 5, 5, 6, 7]
[8, 8, 7, 6, 6, 6, 6, 7]
[9, 9, 8, 7, 7, 7, 7, 7]
[10, 10, 9, 8, 8, 8, 8, 8]
[11, 11, 10, 9, 9, 9, 9, 9]
[0, 1, 2, 3, 4, 5]
[1, 1, 2, 3, 4, 5]
[2, 2, 2, 3, 4, 5]
[3, 3, 3, 3, 4, 5]
[4, 4, 4, 4, 4, 5]
[5, 5, 5, 5, 5, 5]
[6, 6, 6, 6, 6, 6]
[7, 7, 7, 7, 7, 7]
[8, 8, 8, 7, 8, 8]
[9, 9, 9, 8, 8, 9]
[10, 10, 10, 9, 9, 9]
[11, 11, 11, 10, 10, 10]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 0, 1, 2, 3, 4, 5, 6, 7, 8]
[2, 1, 1

[7, 7, 7, 7, 7, 7, 7, 7]
[8, 8, 8, 8, 8, 8, 8, 8]
[9, 9, 9, 9, 9, 9, 8, 9]
[10, 10, 10, 10, 10, 10, 9, 9]
[11, 11, 11, 11, 11, 11, 10, 10]
[12, 12, 12, 12, 12, 12, 11, 11]
[0, 1, 2]
[1, 0, 1]
[2, 1, 1]
[3, 2, 2]
[4, 3, 3]
[5, 4, 4]
[6, 5, 5]
[7, 6, 6]
[8, 7, 7]
[9, 8, 8]
[10, 9, 9]
[11, 10, 10]
[12, 11, 11]
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[10]
[11]
[12]
[0, 1, 2, 3, 4, 5, 6, 7]
[1, 0, 1, 2, 3, 4, 5, 6]
[2, 1, 0, 1, 2, 3, 4, 5]
[3, 2, 1, 0, 1, 2, 3, 4]
[4, 3, 2, 1, 0, 1, 2, 3]
[5, 4, 3, 2, 1, 0, 1, 2]
[6, 5, 4, 3, 2, 1, 0, 1]
[7, 6, 5, 4, 3, 2, 1, 0]
[8, 7, 6, 5, 4, 3, 2, 1]
[9, 8, 7, 6, 5, 4, 3, 2]
[10, 9, 8, 7, 6, 5, 4, 3]
[11, 10, 9, 8, 7, 6, 5, 4]
[12, 11, 10, 9, 8, 7, 6, 5]
[0, 1, 2, 3]
[1, 1, 2, 3]
[2, 2, 2, 3]
[3, 3, 3, 3]
[4, 4, 4, 4]
[5, 5, 5, 5]
[6, 6, 6, 6]
[7, 7, 7, 7]
[8, 8, 8, 8]
[9, 9, 9, 9]
[10, 10, 10, 10]
[11, 11, 11, 11]
[12, 12, 12, 12]
[0, 1, 2, 3, 4]
[1, 1, 2, 3, 3]
[2, 2, 2, 3, 4]
[3, 3, 3, 3, 4]
[4, 4, 3, 4, 4]
[5, 5, 4, 4, 4]
[6, 6, 5, 5, 4]
[7, 7, 6, 6,

[0, 1, 2, 3, 4, 5, 6]
[1, 1, 2, 3, 4, 5, 6]
[2, 2, 2, 3, 4, 5, 6]
[3, 3, 3, 3, 4, 5, 6]
[4, 4, 4, 4, 4, 5, 6]
[5, 5, 5, 5, 5, 5, 5]
[6, 6, 6, 6, 6, 6, 6]
[7, 7, 7, 7, 7, 7, 7]
[8, 8, 8, 8, 8, 8, 8]
[9, 9, 9, 9, 9, 9, 9]
[10, 10, 10, 10, 10, 10, 10]
[0, 1, 2]
[1, 1, 2]
[2, 1, 2]
[3, 2, 2]
[4, 3, 3]
[5, 4, 4]
[6, 5, 5]
[7, 6, 6]
[8, 7, 7]
[9, 8, 8]
[10, 9, 9]
[0, 1, 2, 3, 4]
[1, 1, 2, 3, 4]
[2, 2, 2, 3, 4]
[3, 3, 3, 3, 4]
[4, 4, 4, 4, 4]
[5, 5, 5, 5, 5]
[6, 6, 5, 6, 6]
[7, 7, 6, 5, 6]
[8, 8, 7, 6, 6]
[9, 9, 8, 7, 7]
[10, 10, 9, 8, 8]
[0, 1, 2, 3, 4, 5]
[1, 1, 2, 3, 4, 5]
[2, 2, 2, 3, 4, 5]
[3, 3, 3, 3, 4, 5]
[4, 4, 4, 4, 4, 5]
[5, 5, 5, 5, 5, 5]
[6, 6, 6, 6, 6, 6]
[7, 7, 7, 7, 6, 7]
[8, 8, 8, 8, 7, 7]
[9, 9, 9, 9, 8, 8]
[10, 10, 10, 10, 9, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[1, 1, 2, 3, 4, 5, 6, 7, 8]
[2, 2, 2, 3, 4, 5, 6, 6, 7]
[3, 3, 3, 3, 4, 5, 6, 7, 7]
[4, 4, 4, 4, 4, 5, 6, 6, 7]
[5, 5, 5, 5, 5, 5, 6, 7, 7]
[6, 6, 6, 5, 6, 6, 6, 7, 8]
[7, 7, 6, 6, 6, 7, 7, 7, 8]
[8, 8, 7, 7, 7, 7, 8, 8, 

[0, 1, 2, 3, 4, 5, 6]
[1, 1, 2, 3, 4, 4, 5]
[2, 2, 2, 3, 4, 5, 4]
[3, 3, 3, 3, 4, 5, 5]
[4, 4, 4, 4, 4, 5, 6]
[5, 5, 5, 4, 5, 5, 6]
[6, 6, 6, 5, 5, 6, 6]
[7, 7, 7, 6, 6, 6, 7]
[8, 8, 8, 7, 7, 7, 7]
[0, 1, 2]
[1, 1, 2]
[2, 2, 2]
[3, 3, 3]
[4, 4, 4]
[5, 4, 5]
[6, 5, 4]
[7, 6, 5]
[8, 7, 6]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 1, 2, 3, 4, 5, 6, 7, 7, 8]
[2, 2, 2, 3, 4, 5, 6, 7, 8, 8]
[3, 3, 3, 3, 4, 5, 6, 6, 7, 8]
[4, 4, 4, 4, 4, 5, 6, 7, 7, 8]
[5, 5, 5, 5, 5, 5, 6, 7, 8, 8]
[6, 6, 6, 6, 5, 6, 5, 6, 7, 8]
[7, 6, 7, 7, 6, 5, 6, 6, 7, 8]
[8, 7, 7, 8, 7, 6, 6, 7, 7, 8]
[0, 1, 2]
[1, 1, 2]
[2, 2, 2]
[3, 3, 3]
[4, 4, 4]
[5, 5, 5]
[6, 6, 6]
[7, 6, 7]
[8, 7, 7]
[0, 1, 2, 3, 4, 5, 6, 7]
[1, 1, 2, 3, 4, 5, 6, 7]
[2, 2, 2, 3, 4, 5, 6, 7]
[3, 3, 3, 3, 3, 4, 5, 6]
[4, 4, 4, 4, 4, 4, 5, 6]
[5, 5, 4, 5, 5, 5, 5, 6]
[6, 5, 5, 5, 6, 6, 6, 6]
[7, 6, 6, 6, 6, 6, 7, 7]
[8, 7, 7, 7, 7, 7, 7, 7]
NO
What are the kind of games one can play on windows

what are the kind of games one can play on windows

micros

TRY

[5, 5, 4, 4]
[6, 6, 5, 5]
[7, 7, 6, 6]
[8, 8, 7, 7]
[0, 1, 2, 3, 4]
[1, 1, 2, 3, 4]
[2, 2, 2, 3, 4]
[3, 3, 3, 3, 4]
[4, 4, 4, 3, 4]
[5, 5, 5, 4, 4]
[6, 6, 6, 5, 5]
[7, 7, 7, 6, 5]
[8, 8, 8, 7, 6]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[1, 0, 1, 2, 3, 4, 5, 6, 7]
[2, 1, 0, 1, 2, 3, 4, 5, 6]
[3, 2, 1, 0, 1, 2, 3, 4, 5]
[4, 3, 2, 1, 1, 2, 3, 4, 5]
[5, 4, 3, 2, 1, 2, 3, 4, 5]
[6, 5, 4, 3, 2, 1, 2, 3, 4]
[7, 6, 5, 4, 3, 2, 2, 3, 4]
[8, 7, 6, 5, 4, 3, 3, 3, 4]
[0, 1, 2]
[1, 1, 2]
[2, 2, 1]
[3, 3, 2]
[4, 4, 3]
[5, 5, 4]
[6, 6, 5]
[7, 7, 6]
[8, 8, 7]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 0, 1, 2, 3, 4, 5, 6, 7, 8]
[2, 1, 0, 1, 2, 3, 4, 5, 6, 7]
[3, 2, 1, 1, 2, 3, 4, 5, 6, 7]
[4, 3, 2, 2, 2, 3, 3, 4, 5, 6]
[5, 4, 3, 3, 3, 3, 4, 4, 5, 6]
[6, 5, 4, 4, 4, 4, 4, 5, 5, 6]
[7, 6, 5, 5, 5, 5, 5, 5, 6, 6]
[8, 7, 6, 6, 6, 6, 6, 6, 6, 7]
[0, 1, 2, 3, 4]
[1, 0, 1, 2, 3]
[2, 1, 0, 1, 2]
[3, 2, 1, 1, 2]
[4, 3, 2, 2, 2]
[5, 4, 3, 3, 3]
[6, 5, 4, 4, 4]
[7, 6, 5, 5, 5]
[8, 7, 6, 6, 6]
[0, 1, 2, 3, 4, 5]
[1, 1, 2, 3, 3, 4]
[

...

who are starring in the movies written by Mark Sandrich 

who are starring in the movies written by mark sandrich 

film

TRYING
[0, 1, 2, 3]
[1, 1, 2, 3]
[2, 2, 2, 3]
[3, 3, 3, 3]
[4, 4, 4, 4]
[5, 5, 5, 5]
[0, 1, 2, 3]
[1, 1, 2, 3]
[2, 2, 2, 3]
[3, 3, 3, 3]
[4, 4, 4, 4]
[5, 5, 5, 5]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[1, 1, 2, 3, 4, 5, 6, 7, 8]
[2, 2, 2, 3, 4, 5, 5, 6, 7]
[3, 3, 3, 3, 4, 5, 6, 6, 7]
[4, 4, 4, 4, 4, 5, 6, 7, 7]
[5, 5, 5, 5, 5, 5, 6, 7, 8]
[0, 1, 2]
[1, 1, 2]
[2, 1, 2]
[3, 2, 2]
[4, 3, 3]
[5, 4, 4]
[0, 1, 2, 3]
[1, 1, 2, 3]
[2, 2, 2, 3]
[3, 3, 3, 3]
[4, 4, 4, 4]
[5, 5, 5, 5]
[0, 1, 2, 3, 4, 5, 6]
[1, 1, 2, 3, 4, 5, 6]
[2, 2, 2, 3, 3, 4, 5]
[3, 3, 3, 3, 4, 4, 5]
[4, 3, 4, 4, 4, 5, 5]
[5, 4, 4, 5, 5, 5, 6]
[0, 1, 2, 3, 4, 5, 6, 7]
[1, 1, 2, 3, 4, 5, 6, 7]
[2, 2, 2, 2, 3, 4, 5, 6]
[3, 3, 3, 3, 3, 4, 5, 6]
[4, 4, 4, 4, 4, 4, 5, 6]
[5, 5, 5, 5, 5, 5, 5, 6]
[0, 1, 2]
[1, 1, 2]
[2, 2, 2]
[3, 3, 3]
[4, 4, 4]
[5, 5, 5]
[0, 1, 2, 3, 4]
[1, 1, 2, 3, 4]
[2, 2, 2, 3, 4]
[3, 3, 3, 3, 4]
[4,

[1, 1, 1, 2, 3, 4, 5, 6, 7, 8]
[2, 2, 2, 2, 3, 4, 5, 6, 7, 8]
[3, 3, 3, 3, 3, 3, 4, 5, 6, 7]
[4, 4, 4, 4, 4, 4, 4, 5, 6, 7]
[5, 5, 5, 5, 5, 5, 5, 5, 5, 6]
[6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
[7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
[8, 8, 8, 8, 8, 8, 7, 8, 8, 8]
[9, 9, 9, 9, 9, 9, 8, 8, 9, 9]
[10, 10, 10, 10, 10, 10, 9, 9, 9, 10]
[11, 11, 11, 11, 11, 11, 10, 10, 10, 10]
[12, 12, 12, 12, 12, 12, 11, 11, 10, 11]
[13, 13, 13, 13, 13, 13, 12, 12, 11, 11]
[14, 14, 14, 14, 14, 14, 13, 13, 12, 12]
[0, 1, 2, 3, 4]
[1, 1, 2, 3, 4]
[2, 2, 2, 3, 4]
[3, 3, 3, 3, 4]
[4, 4, 4, 4, 4]
[5, 5, 5, 5, 4]
[6, 6, 6, 6, 5]
[7, 7, 7, 7, 6]
[8, 8, 8, 8, 7]
[9, 9, 9, 9, 8]
[10, 10, 9, 10, 9]
[11, 11, 10, 10, 10]
[12, 12, 11, 11, 10]
[13, 13, 12, 12, 11]
[14, 14, 13, 13, 12]
[0, 1, 2, 3, 4]
[1, 1, 2, 3, 4]
[2, 2, 2, 3, 3]
[3, 3, 3, 3, 4]
[4, 4, 4, 4, 4]
[5, 5, 4, 4, 5]
[6, 6, 5, 5, 5]
[7, 7, 6, 6, 6]
[8, 8, 7, 7, 7]
[9, 9, 8, 8, 8]
[10, 10, 9, 9, 9]
[11, 11, 10, 10, 10]
[12, 12, 11, 10, 11]
[13, 13, 12, 11, 11]
[14, 14, 13, 12

[0, 1, 2, 3, 4, 5, 6]
[1, 1, 2, 3, 4, 5, 6]
[2, 2, 2, 3, 3, 4, 5]
[3, 3, 3, 3, 4, 4, 5]
[4, 3, 4, 4, 4, 5, 5]
[5, 4, 4, 5, 5, 5, 6]
[0, 1, 2, 3, 4, 5]
[1, 1, 2, 3, 4, 5]
[2, 2, 2, 3, 4, 5]
[3, 3, 3, 3, 4, 5]
[4, 4, 4, 4, 4, 5]
[5, 5, 5, 5, 5, 5]
[0, 1, 2, 3, 4, 5, 6]
[1, 1, 2, 3, 4, 5, 6]
[2, 2, 2, 2, 3, 4, 5]
[3, 3, 3, 3, 3, 4, 5]
[4, 4, 4, 4, 4, 4, 5]
[5, 5, 5, 5, 5, 5, 5]
[0, 1, 2]
[1, 1, 2]
[2, 1, 2]
[3, 2, 2]
[4, 3, 3]
[5, 4, 4]
[0, 1, 2, 3, 4]
[1, 1, 2, 3, 4]
[2, 2, 2, 3, 4]
[3, 3, 3, 3, 4]
[4, 3, 4, 4, 4]
[5, 4, 4, 5, 5]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[1, 1, 2, 3, 4, 5, 6, 7, 8]
[2, 2, 2, 3, 4, 5, 6, 7, 8]
[3, 3, 3, 3, 4, 5, 6, 7, 8]
[4, 4, 4, 4, 4, 5, 6, 7, 8]
[5, 5, 5, 5, 5, 5, 6, 7, 7]
NO
What ethnicity do people in British Columbia belong to

what ethnicity do people in british columbia belong to

pers

TRYING
[0, 1, 2, 3, 4]
[1, 1, 2, 3, 4]
[2, 2, 2, 3, 4]
[3, 3, 3, 3, 4]
[4, 4, 4, 4, 4]
[5, 5, 5, 5, 5]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[2, 1, 2, 3,

[4, 4, 4, 4]
[5, 5, 5, 5]
[6, 6, 6, 6]
[7, 7, 7, 7]
[8, 8, 8, 8]
[9, 9, 9, 9]
[10, 10, 10, 10]
[0, 1, 2, 3, 4, 5, 6]
[1, 1, 2, 3, 4, 5, 6]
[2, 2, 2, 3, 4, 5, 6]
[3, 3, 3, 3, 4, 5, 6]
[4, 4, 4, 4, 4, 5, 6]
[5, 5, 5, 5, 5, 5, 5]
[6, 6, 6, 6, 6, 6, 6]
[7, 7, 7, 7, 7, 7, 7]
[8, 8, 8, 8, 8, 8, 8]
[9, 9, 9, 9, 9, 9, 9]
[10, 10, 10, 10, 10, 10, 10]
[0, 1, 2]
[1, 1, 2]
[2, 1, 2]
[3, 2, 2]
[4, 3, 3]
[5, 4, 4]
[6, 5, 5]
[7, 6, 6]
[8, 7, 7]
[9, 8, 8]
[10, 9, 9]
[0, 1, 2, 3]
[1, 1, 2, 3]
[2, 2, 2, 3]
[3, 3, 3, 3]
[4, 4, 4, 4]
[5, 4, 5, 5]
[6, 5, 5, 6]
[7, 6, 6, 6]
[8, 7, 7, 7]
[9, 8, 8, 8]
[10, 9, 9, 9]
[0, 1, 2, 3, 4, 5, 6]
[1, 1, 2, 3, 4, 5, 6]
[2, 2, 2, 3, 4, 5, 6]
[3, 3, 3, 3, 4, 4, 5]
[4, 4, 4, 4, 4, 5, 5]
[5, 5, 5, 4, 4, 5, 6]
[6, 6, 5, 5, 5, 5, 6]
[7, 7, 6, 6, 6, 6, 6]
[8, 8, 7, 7, 7, 7, 7]
[9, 9, 8, 8, 8, 8, 8]
[10, 10, 9, 9, 9, 9, 9]
[0, 1, 2]
[1, 1, 2]
[2, 2, 2]
[3, 3, 3]
[4, 4, 4]
[5, 5, 4]
[6, 5, 5]
[7, 6, 6]
[8, 7, 7]
[9, 8, 8]
[10, 9, 9]
[0, 1, 2, 3, 4, 5, 6]
[1, 1, 2, 3, 4, 5, 6]
[2

[5, 5, 5, 5, 5, 5, 6, 6, 7]
[6, 6, 6, 6, 6, 6, 6, 7, 7]
[7, 7, 7, 7, 7, 7, 7, 7, 8]
[8, 8, 8, 8, 8, 8, 8, 8, 8]
[9, 9, 9, 9, 9, 9, 9, 9, 9]
[10, 10, 10, 10, 10, 10, 10, 10, 10]
[11, 11, 11, 11, 11, 11, 11, 11, 11]
[12, 12, 12, 12, 12, 12, 12, 12, 12]
[13, 13, 13, 12, 13, 13, 13, 13, 13]
[14, 14, 14, 13, 13, 14, 14, 13, 14]
[15, 15, 15, 14, 14, 14, 15, 14, 14]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[1, 1, 2, 3, 4, 5, 6, 7, 8]
[2, 2, 2, 3, 4, 4, 5, 6, 7]
[3, 3, 3, 3, 4, 5, 5, 6, 7]
[4, 4, 4, 4, 4, 5, 6, 6, 7]
[5, 5, 5, 5, 5, 5, 6, 7, 7]
[6, 6, 6, 6, 6, 6, 5, 6, 7]
[7, 7, 7, 7, 7, 7, 6, 6, 7]
[8, 8, 8, 8, 8, 8, 7, 7, 7]
[9, 9, 9, 9, 9, 9, 8, 8, 8]
[10, 10, 10, 10, 10, 10, 9, 9, 9]
[11, 11, 11, 11, 11, 10, 10, 10, 10]
[12, 12, 12, 12, 12, 11, 11, 11, 11]
[13, 13, 13, 13, 13, 12, 12, 12, 12]
[14, 14, 14, 14, 14, 13, 13, 13, 13]
[15, 15, 15, 15, 15, 14, 14, 14, 14]
[0, 1, 2, 3]
[1, 1, 2, 3]
[2, 2, 2, 2]
[3, 3, 2, 3]
[4, 4, 3, 3]
[5, 4, 4, 4]
[6, 5, 5, 5]
[7, 6, 6, 6]
[8, 7, 7, 7]
[9, 8, 8, 8]
[10, 9, 9

In [5]:
print(len(matched6), len(notmatched))

42 6784
